<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMSkillsNetworkBD0231ENCoursera2789-2023-01-01">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


## Model persistence using SparkML


Estimated time needed: **30** minutes


<p style='color: red'>The purpose of this lab is to show you how to use SparkML to persist a model and to load the persisted model.


## __Table of Contents__

<ol>
  <li>
    <a href="#Objectives">Objectives
    </a>
  </li>
  <li>
    <a href="#Datasets">Datasets
    </a>
  </li>
  <li>
    <a href="#Setup">Setup
    </a>
    <ol>
      <li>
        <a href="#Installing-Required-Libraries">Installing Required Libraries
        </a>
      </li>
      <li>
        <a href="#Importing-Required-Libraries">Importing Required Libraries
        </a>
      </li>
    </ol>
  </li>
  <li>
    <a href="#Examples">Examples
    </a>
    <ol>
    <li>
      <a href="#Task-1---Create-a-model">Task 1 - Create a model
      </a>
    </li>
    <li>
      <a href="#Task-2---Save-the-model">Task 2 - Save the model
      </a>
    </li>
    <li>
      <a href="#Task-3---Load-the-model">Task 3 - Load the model
      </a>
    </li>
    <li>
      <a href="#Task-4---Predict-using-the-loaded-model">Task 4 - Predict using the loaded model
      </a>
    </li>
    </ol>
  </li>
  <li>
    <a href="#Exercises">Exercises
    </a>
  </li>
  <ol>
    <li>
      <a href="#Exercise-1---Create-a-model">Exercise 1 - Create a model
      </a>
    </li>
    <li>
      <a href="#Exercise-2---Save-the-model">Exercise 2 - Save the model
      </a>
    </li>
    <li>
      <a href="#Exercise-3---Load-the-model">Exercise 3 - Load the model
      </a>
    </li>
    <li>
      <a href="#Exercise-4---Predict-using-the-loaded-model">Exercise 4 - Predict using the loaded model
      </a>
    </li>
  </ol>
</ol>


## Objectives

After completing this lab you will be able to:

 - Save a trained model.
 - Load a saved model.
 - Make predictions using the loaded model.


## Datasets

In this lab you will be using dataset(s):

 - Modified version of car mileage dataset. Original dataset available at https://archive.ics.uci.edu/ml/datasets/auto+mpg 
 - Modified version of diamonds dataset. Original dataset available at https://www.openml.org/search?type=data&sort=runs&id=42225&status=active
 


----


## Setup


For this lab, we will be using the following libraries:

*   [`PySpark`](https://spark.apache.org/docs/latest/api/python/index.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMSkillsNetworkBD0231ENCoursera2789-2023-01-01) for connecting to the Spark Cluster


### Installing Required Libraries

Spark Cluster is pre-installed in the Skills Network Labs environment. However, you need libraries like pyspark and findspark to connect to this cluster.

If you wish to download this jupyter notebook and run on your local computer, follow the instructions mentioned <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/labs/Connecting_to_spark_cluster_using_Skills_Network_labs.ipynb">here.</a>



The following required libraries are __not__ pre-installed in the Skills Network Labs environment. __You will need to run the following cell__ to install them:


In [ ]:
# !pip install pyspark==3.1.2 -q
# !pip install findspark -q

### Importing Required Libraries

_We recommend you import all required libraries in one place (here):_


In [1]:
# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

# FindSpark simplifies the process of using Apache Spark with Python

import findspark
findspark.init()

#import functions/Classes for sparkml

from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline

# import functions/Classes for metrics
from pyspark.ml.evaluation import RegressionEvaluator


# Examples


## Task 1 - Create a model


In [2]:
#Create SparkSession
#Ignore any warnings by SparkSession command

spark = SparkSession.builder.appName("Model Persistence").getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/04/19 12:27:23 WARN Utils: Your hostname, DESKTOP-LMLK3KO resolves to a loopback address: 127.0.1.1; using 172.19.128.203 instead (on interface eth0)
24/04/19 12:27:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/19 12:27:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Download the data file


In [ ]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0231EN-SkillsNetwork/datasets/mpg.csv


Load the dataset into the spark dataframe


In [3]:
# using the spark.read.csv function we load the data into a dataframe.
# the header = True mentions that there is a header row in out csv file
# the inferSchema = True, tells spark to automatically find out the data types of the columns.

# Load mpg dataset
mpg_data = spark.read.csv("mpg.csv", header=True, inferSchema=True)


Print the schema of the dataset


In [4]:
mpg_data.printSchema()

root
 |-- MPG: double (nullable = true)
 |-- Cylinders: integer (nullable = true)
 |-- Engine Disp: double (nullable = true)
 |-- Horsepower: integer (nullable = true)
 |-- Weight: integer (nullable = true)
 |-- Accelerate: double (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Origin: string (nullable = true)



Show top 5 rows from the dataset


In [5]:
mpg_data.show(5)

+----+---------+-----------+----------+------+----------+----+--------+
| MPG|Cylinders|Engine Disp|Horsepower|Weight|Accelerate|Year|  Origin|
+----+---------+-----------+----------+------+----------+----+--------+
|15.0|        8|      390.0|       190|  3850|       8.5|  70|American|
|21.0|        6|      199.0|        90|  2648|      15.0|  70|American|
|18.0|        6|      199.0|        97|  2774|      15.5|  70|American|
|16.0|        8|      304.0|       150|  3433|      12.0|  70|American|
|14.0|        8|      455.0|       225|  3086|      10.0|  70|American|
+----+---------+-----------+----------+------+----------+----+--------+
only showing top 5 rows



We ask the VectorAssembler to group a bunch of inputCols as single column named "features"


In [6]:
# Prepare feature vector
assembler = VectorAssembler(inputCols=["Cylinders", "Engine Disp", "Horsepower", "Weight", "Accelerate", "Year"], outputCol="features")
mpg_transformed_data = assembler.transform(mpg_data)


Display the assembled "features" and the label column "MPG"


In [7]:
mpg_transformed_data.select("features","MPG").show()

+--------------------+----+
|            features| MPG|
+--------------------+----+
|[8.0,390.0,190.0,...|15.0|
|[6.0,199.0,90.0,2...|21.0|
|[6.0,199.0,97.0,2...|18.0|
|[8.0,304.0,150.0,...|16.0|
|[8.0,455.0,225.0,...|14.0|
|[8.0,350.0,165.0,...|15.0|
|[8.0,307.0,130.0,...|18.0|
|[8.0,454.0,220.0,...|14.0|
|[8.0,400.0,150.0,...|15.0|
|[8.0,307.0,200.0,...|10.0|
|[8.0,383.0,170.0,...|15.0|
|[8.0,318.0,210.0,...|11.0|
|[8.0,360.0,215.0,...|10.0|
|[8.0,429.0,198.0,...|15.0|
|[6.0,200.0,85.0,2...|21.0|
|[8.0,302.0,140.0,...|17.0|
|[8.0,304.0,193.0,...| 9.0|
|[8.0,340.0,160.0,...|14.0|
|[6.0,198.0,95.0,2...|22.0|
|[8.0,440.0,215.0,...|14.0|
+--------------------+----+
only showing top 20 rows



We split the data set in the ratio of 70:30. 70% training data, 30% testing data.


In [8]:
# Split data into training and testing sets
(training_data, testing_data) = mpg_transformed_data.randomSplit([0.7, 0.3])


Create a LR model and train the model using the pipeline on training data set


In [9]:
# Train linear regression model
# Ignore any warnings
lr = LinearRegression(labelCol="MPG", featuresCol="features")
pipeline = Pipeline(stages=[lr])
model = pipeline.fit(training_data)


24/04/19 12:35:20 WARN Instrumentation: [80b1e30f] regParam is zero, which might cause numerical instability and overfitting.
24/04/19 12:35:21 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/04/19 12:35:21 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


## Task 2 - Save the model


Create a folder where the model will to be saved


In [10]:
!mkdir model_storage

In [11]:
# Persist the model to the path "./model_stoarage/"

model.write().overwrite().save("./model_storage/")

#The overwrite method is used to overwrite the model if it already exists,
#and the save method is used to specify the path where the model should be saved.



## Task 3 - Load the model


In [12]:
from pyspark.ml.pipeline import PipelineModel

# Load persisted model
loaded_model = PipelineModel.load("./model_storage/")

## Task 4 - Predict using the loaded model


In [13]:
# Make predictions on test data
predictions = loaded_model.transform(testing_data)
#In the above example, we use the load method of the PipelineModel object to load the persisted model from disk. We can then use this loaded model to make predictions on new data using the transform method.


Your model is now trained. We use the testing data to make predictions.


In [14]:
# Make predictions on testing data
predictions = model.transform(testing_data)

In [15]:
predictions.select("prediction").show(5)

+------------------+
|        prediction|
+------------------+
| 7.454420605202602|
|  8.06236246977088|
| 21.20209686286495|
|  17.5813043604794|
|11.789769182677867|
+------------------+
only showing top 5 rows



Stop Spark Session


In [16]:
spark.stop()

# Exercises


### Exercise 1 - Create a model


Create a spark session with appname "Model Persistence Exercise"


In [17]:
spark = SparkSession.builder.appName("Model Persistence Exercise").getOrCreate()

<details>
    <summary>Click here for a Hint</summary>
    
Use the SparkSession.builder

</details>


<details>
    <summary>Click here for Solution</summary>

```python
spark = SparkSession.builder.appName("Model Persistence Exercise").getOrCreate()
```

</details>


Download the data set


In [ ]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0231EN-SkillsNetwork/datasets/diamonds.csv


Load the dataset into a spark dataframe


In [18]:
diamond_data = spark.read.csv("diamonds.csv", header=True, inferSchema=True)

<details>
    <summary>Click here for a Hint</summary>
    
Use the spark.read.csv function

</details>


<details>
    <summary>Click here for Solution</summary>

```python
diamond_data = spark.read.csv("diamonds.csv", header=True, inferSchema=True)
```
</details>


Display sample data from dataset


In [19]:
diamond_data.show(5)

+---+-----+-------+-----+-------+-----+-----+-----+----+----+----+
|  s|carat|    cut|color|clarity|depth|table|price|   x|   y|   z|
+---+-----+-------+-----+-------+-----+-----+-----+----+----+----+
|  1| 0.23|  Ideal|    E|    SI2| 61.5| 55.0|  326|3.95|3.98|2.43|
|  2| 0.21|Premium|    E|    SI1| 59.8| 61.0|  326|3.89|3.84|2.31|
|  3| 0.23|   Good|    E|    VS1| 56.9| 65.0|  327|4.05|4.07|2.31|
|  4| 0.29|Premium|    I|    VS2| 62.4| 58.0|  334| 4.2|4.23|2.63|
|  5| 0.31|   Good|    J|    SI2| 63.3| 58.0|  335|4.34|4.35|2.75|
+---+-----+-------+-----+-------+-----+-----+-----+----+----+----+
only showing top 5 rows



Assemble the columns columns carat,depth and table into a single column named "features"


In [20]:
assembler = VectorAssembler(inputCols=["carat", "depth", "table"], outputCol="features")
diamond_transformed_data = assembler.transform(diamond_data)

<details>
    <summary>Click here for a Hint</summary>
    
Refer to Task1
</details>


<details>
    <summary>Click here for Solution</summary>

```python
assembler = VectorAssembler(inputCols=["carat", "depth", "table"], outputCol="features")
diamond_transformed_data = assembler.transform(diamond_data)
```

</details>


Print the vectorized features and label columns


In [21]:
diamond_transformed_data.select("features","price").show()

+----------------+-----+
|        features|price|
+----------------+-----+
|[0.23,61.5,55.0]|  326|
|[0.21,59.8,61.0]|  326|
|[0.23,56.9,65.0]|  327|
|[0.29,62.4,58.0]|  334|
|[0.31,63.3,58.0]|  335|
|[0.24,62.8,57.0]|  336|
|[0.24,62.3,57.0]|  336|
|[0.26,61.9,55.0]|  337|
|[0.22,65.1,61.0]|  337|
|[0.23,59.4,61.0]|  338|
| [0.3,64.0,55.0]|  339|
|[0.23,62.8,56.0]|  340|
|[0.22,60.4,61.0]|  342|
|[0.31,62.2,54.0]|  344|
| [0.2,60.2,62.0]|  345|
|[0.32,60.9,58.0]|  345|
| [0.3,62.0,54.0]|  348|
| [0.3,63.4,54.0]|  351|
| [0.3,63.8,56.0]|  351|
| [0.3,62.7,59.0]|  351|
+----------------+-----+
only showing top 20 rows



Split the dataset into training and testing sets in the ratio of 70:30.


In [22]:
(training_data, testing_data) =  diamond_transformed_data.randomSplit([0.7, 0.3])

<details>
    <summary>Click here for a Hint</summary>
    
use the randomSplit method
</details>


<details>
    <summary>Click here for Solution</summary>

```
(training_data, testing_data) = diamond_transformed_data.randomSplit([0.7, 0.3])


```

</details>


Create a LR model and train the model using the pipeline on training data set


In [23]:
# Train linear regression model
# Ignore any warnings
lr =  LinearRegression(featuresCol="features", labelCol="price")
pipeline =  Pipeline(stages=[lr])
model =  pipeline.fit(training_data)


24/04/19 12:58:43 WARN Instrumentation: [124be671] regParam is zero, which might cause numerical instability and overfitting.


<details>
    <summary>Click here for a Hint</summary>
    
use the Pipeline with an lr stage
</details>


<details>
    <summary>Click here for Solution</summary>

```
lr = LinearRegression(labelCol="price", featuresCol="features")
pipeline = Pipeline(stages=[lr])
model = pipeline.fit(training_data)

```

</details>


### Exercise 2 - Save the model


Create a folder "diamond_model". This is where the model will to be saved


In [24]:
!mkdir diamond_model

Persist the model to the folder "diamond_model"


In [25]:
#your code goes here
model.write().overwrite().save("./diamond_model/")

<details>
    <summary>Click here for a Hint</summary>
    
use the write method of the model
</details>


<details>
    <summary>Click here for Solution</summary>

```
model.write().overwrite().save("./diamond_model/")

```

</details>


### Exercise 3 - Load the model


Load the model from the folder "diamond_model"


In [26]:
from pyspark.ml.pipeline import PipelineModel

# Load persisted model
loaded_model = PipelineModel.load("./diamond_model/")

<details>
    <summary>Click here for a Hint</summary>
    
use the load method of the PipelineModel
</details>


<details>
    <summary>Click here for Solution</summary>

```
loaded_model = PipelineModel.load("./diamond_model/")
```

</details>


### Exercise 4 - Predict using the loaded model


Make predictions on test data


In [27]:

predictions = loaded_model.transform(testing_data)

<details>
    <summary>Click here for a Hint</summary>
    
use the transform method of the loaded model
</details>


<details>
    <summary>Click here for Solution</summary>

```
predictions = loaded_model.transform(testing_data)

```

</details>


In [28]:
predictions.select("prediction").show(5)

+------------------+
|        prediction|
+------------------+
|-588.1996654460072|
| -557.837144170142|
| 340.1646908012626|
|-317.1935626118702|
|-83.99487075095203|
+------------------+
only showing top 5 rows



Stop Spark Session


In [29]:
spark.stop()

Congratulations you have completed this lab.<br>


## Authors


[Ramesh Sannareddy](https://www.linkedin.com/in/rsannareddy/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMBD0231ENSkillsNetwork866-2023-01-01)


### Other Contributors


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2023-05-04|0.1|Ramesh Sannareddy|Initial Version Created|


Copyright © 2023 IBM Corporation. All rights reserved.
